In [1]:
import sys
sys.path.append("./utils")
from utils.agents import *
from utils.embedding import *
from utils.get_E import *
from utils.json_extract import *
from utils.R import *

In [2]:
#GPT-API-KEY and GLM-API-KEY

api_key_gpt = "sk-pOcjk3qopBxmFcGJ464eFc942f7245B68e076821F959C7B5"
api_key_glm = "3a96f3438411dfd81700ce172c4e5b89.fSLyWokcvYectqvI"

### Building a Knowledge Graph from a txt

#### Here, we use the paper as the material, refer to [India's Opportunity to Address Human Resource Challenges](https://pubmed.ncbi.nlm.nih.gov/37448434/#:~:text=The%20paucity%20of%20skilled%20personnel%20must%20be%20addressed,budget%20offers%20an%20unprecedented%20opportunity%20to%20do%20this)

In [3]:
# graph-task and external corpus

try:
    with open('./prompts/graph.txt', 'r', encoding='utf-8') as file:
        KG_task  = file.read()
except FileNotFoundError:
    res = "File not found."

try:
    with open("./datasets/cureus-0015-00000040274.txt", 'r', encoding='utf-8') as file:
        document_content = file.read()
except FileNotFoundError:
    document_content = "The specified file path was not found."

In [4]:
# create a knowledge graph using GPT-4o

import http.client
import json

# Initialize the connection
conn = http.client.HTTPSConnection("35fast.aigcbest.top")
payload = json.dumps({
   "model": "gpt-4o",
   "messages": [
      {
         "role": "user",
         "content": KG_task + 'the given passage is: ' + document_content
      }
   ]
})
headers = {
   'Accept': 'application/json',
   'Authorization': f'Bearer {api_key_gpt}',  # Make sure api_key_gpt is defined earlier
   'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
   'Content-Type': 'application/json'
}

# Send the POST request
conn.request("POST", "/v1/chat/completions", payload, headers)
res = conn.getresponse()
data = res.read()

# Decode the response and load it as a dictionary
response = json.loads(data.decode("utf-8"))

# Extract the content part from the response
content = response['choices'][0]['message']['content']

# Print the extracted content
print(content)

```json
{
    "nodes": [
        {"id": 1, "name": "India", "explanation": "India has a population of 1.3 billion and a density of doctors and nurses lower than the WHO threshold."},
        {"id": 2, "name": "Health Indicators", "explanation": "India's health indicators have improved but continue to lag behind peer nations."},
        {"id": 3, "name": "WHO Threshold", "explanation": "The WHO threshold is 44.5 doctors, nurses, and midwives per 10,000 population."},
        {"id": 4, "name": "Human Resources", "explanation": "Human resources have a central role in healthcare but show a paucity in skilled personnel."},
        {"id": 5, "name": "Universal Health Coverage", "explanation": "Universal health coverage is a goal India must accelerate towards for sustainable development."},
        {"id": 6, "name": "National Medical Council", "explanation": "The newly minted National Medical Council has opportunities within healthcare to achieve SDGs."},
        {"id": 7, "name": "Public Sec

In [5]:
# extract the json data from the response

extracted_json = clean_and_extract_json(content)
extracted_json

{'nodes': [{'id': 1,
   'name': 'India',
   'explanation': 'India has a population of 1.3 billion and a density of doctors and nurses lower than the WHO threshold.'},
  {'id': 2,
   'name': 'Health Indicators',
   'explanation': "India's health indicators have improved but continue to lag behind peer nations."},
  {'id': 3,
   'name': 'WHO Threshold',
   'explanation': 'The WHO threshold is 44.5 doctors, nurses, and midwives per 10,000 population.'},
  {'id': 4,
   'name': 'Human Resources',
   'explanation': 'Human resources have a central role in healthcare but show a paucity in skilled personnel.'},
  {'id': 5,
   'name': 'Universal Health Coverage',
   'explanation': 'Universal health coverage is a goal India must accelerate towards for sustainable development.'},
  {'id': 6,
   'name': 'National Medical Council',
   'explanation': 'The newly minted National Medical Council has opportunities within healthcare to achieve SDGs.'},
  {'id': 7,
   'name': 'Public Sector',
   'explanati

In [6]:
# Store this json

output_path = './knowledge_graph/cureus-0015-00000040274.json'

try:
    with open(output_path, 'w', encoding='utf-8') as json_file:
        json.dump(extracted_json, json_file, ensure_ascii=False, indent=4)
    res = "File saved successfully."
except Exception as e:
    res = f"An error occurred: {e}"

### Retrieve From this KG using a query

In [7]:
# user query
query = 'How can the National Medical Council (NMC) regulate the growing divide between the public and private health sectors in India, particularly in light of the increasing reliance on Auxiliary Nurse Midwives (ANMs) and Accredited Social Health Activists (ASHAs) for rural healthcare delivery, while addressing the broader challenges of digital technology adoption and health infrastructure development as outlined in the National Health Mission (NHM)?'


In [8]:
# get a map: id and similarity value

ref_vec = get_embedding(api_key_glm, query)
id_list0 = []

for i in range(len(extracted_json["nodes"])):
    # Get the embedding for the current node's name
    embedding = get_embedding(key=api_key_glm, input_text=extracted_json["nodes"][i]["name"])
    
    # Calculate the similarity between the embedding and the reference vector
    sim = cosine_similarity(ref_vec, embedding)
    
    # Append the id and similarity as a dictionary to the list
    id_list0.append({
        "id": extracted_json["nodes"][i]["id"],
        "similarity": sim
    })


In [9]:
# get the top-beta_1 id and sort them in descending order

bata_1 = 3
sorted_list = sorted(id_list0, key=lambda x: x['similarity'], reverse=True)
id_list = [item['id'] for item in sorted_list[:bata_1]]

id_list

[6, 12, 10]

In [10]:
# get the retrieved result

data = get_E(extracted_json)
alpha_1 = 2
alpha_2 = 2
retrieved_res = R(data, id_list, alpha_1, alpha_2)
retrieved_res

{'nodes_retrieved': [{'id': 6,
   'name': 'National Medical Council',
   'explanation': 'The newly minted National Medical Council has opportunities within healthcare to achieve SDGs.',
   'length': 0},
  {'id': 12,
   'name': 'Auxiliary Nurse Midwives (ANM)',
   'explanation': 'ANMs are meant to serve a population of 5,000 people but often serve up to 20,000.',
   'length': 0},
  {'id': 10,
   'name': 'National Health Mission',
   'explanation': 'NHM has aimed to increase the quantum of services since its inception in 2005.',
   'length': 0},
  {'id': 8,
   'name': 'Private Sector',
   'explanation': 'The private sector is the dominant provider, perceived to provide quality care and involves booming medical tourism.',
   'length': 1},
  {'id': 11,
   'name': 'Accredited Social Health Activists (ASHA)',
   'explanation': 'ASHAs are community health workers serving to extend the reach of Auxiliary Nurse Midwives.',
   'length': 1},
  {'id': 7,
   'name': 'Public Sector',
   'explanation

### Construct multi agents provided for users

In [11]:
# construct tasks set and llms set

file_path = './prompts/'
dirs = ['agent1.txt', 'agent2.txt', 'agent3.txt', 'agent4.txt', 'fusion.txt']

# Reading the content of files and storing them in the 'tasks' dictionary directly
tasks = {key: open(f"{file_path}{key}", 'r', encoding='utf-8').read() for key in dirs}

# Renaming the keys in the 'tasks' dictionary
tasks = {
    'concepts': tasks['agent1.txt'],
    'hint': tasks['agent2.txt'],
    'assess': tasks['agent3.txt'],
    'questions': tasks['agent4.txt'],
    'fusion': tasks['fusion.txt']
}

agents = {}
llms_list = ["glm-4", "glm-4-plus", "glm-4-long", "glm-4-0520", "glm-4-flash"]

# Loop to initialize Agent instances for each LLM and task content, and create a dictionary with LLM+task as keys
for i in llms_list:
    for task_name, task_content in tasks.items():
        key = f"{i}_{task_name}"  # Create a key by combining LLM model name and task name
        agents[key] = Agent(api_key=api_key_glm, model_name=i, system_content=task_content)

#### Testing an Agent

In [12]:
a0 = agents["glm-4-plus_concepts"]
user_input = """
What does the National Medical Council (NMC) mean? less than 100 words.
"""
res0 = a0.get_response(user_content=user_input)['response_content']
print("The reponse is" + " \n" + res0)

The reponse is 
The National Medical Council (NMC) is an apex regulatory body in India overseeing medical education and practice. It replaced the Medical Council of India (MCI) to enhance transparency and accountability. The NMC regulates medical colleges, sets standards for medical education, and maintains the Indian Medical Register. It aims to ensure quality healthcare by accrediting medical institutions and certifying medical professionals. The council also addresses grievances and enforces ethical practices in the medical field.


### Choose 5 agents

In [13]:
agent1 = agents["glm-4-flash_concepts"]  # Responsible for handling the 'concepts' task using the 'glm-4-flash' model
agent2 = agents["glm-4-plus_hint"]       # Handles the 'hint' task using the 'glm-4-plus' model
agent3 = agents["glm-4-plus_assess"]     # Manages the 'assess' task using the 'glm-4-plus' model
agent4 = agents["glm-4-0520_questions"]  # Processes the 'questions' task using the 'glm-4-0520' model
agent5 = agents["glm-4-flash_fusion"]    # Takes care of the 'fusion' task using the 'glm-4-flash' model

### Generate responses and Fuse them

In [14]:
# generate responses


retrieved_res= json.dumps(retrieved_res, ensure_ascii=False, indent=2)
agents_parallel = [agent1, agent2, agent3]

input = 'my question is: /n' + query +  \
'please answer my question based on the kwnoledge graph content as follows' + \
retrieved_res

response = ""

for agent in agents_parallel:
    res = agent.get_response(user_content=input)['response_content']
    response += res

In [15]:
response

'The concepts and explanations are given as follows:\n\n{\n  "term0": {\n    "term": "National Medical Council (NMC)",\n    "explanation": "The National Medical Council has opportunities within healthcare to achieve Sustainable Development Goals (SDGs)."\n  },\n  "term1": {\n    "term": "Auxiliary Nurse Midwives (ANM)",\n    "explanation": "ANMs are meant to serve a population of 5,000 people but often serve up to 20,000, indicating a potential overstretch in their roles."\n  },\n  "term2": {\n    "term": "National Health Mission (NHM)",\n    "explanation": "NHM has aimed to increase the quantum of services since its inception in 2005, focusing on improving public healthcare in India."\n  },\n  "term3": {\n    "term": "Private Sector",\n    "explanation": "The private sector is the dominant provider of healthcare services in India, offering quality care and attracting medical tourism."\n  },\n  "term4": {\n    "term": "Accredited Social Health Activists (ASHA)",\n    "explanation": "AS

In [16]:
# fuse the responses from agent1-3.

answer_fusion = agent5.get_response(user_content=response)['response_content']
print(answer_fusion)

Based on your question, here is the information I have gathered:

1. Extracted Terminology/Concepts and Explanations:
   - National Medical Council (NMC): An organization within healthcare that has opportunities to achieve Sustainable Development Goals (SDGs).
   - Auxiliary Nurse Midwives (ANM): Healthcare professionals meant to serve a population of 5,000 but often stretched to serve up to 20,000, indicating a potential overstretch in their roles.
   - National Health Mission (NHM): A mission since 2005 aimed at increasing the quantum of services, focusing on improving public healthcare in India.
   - Private Sector: Dominant provider of healthcare services in India, offering quality care and attracting medical tourism.
   - Accredited Social Health Activists (ASHA): Community health workers that extend the reach of ANMs, increasing the accessibility of healthcare services.
   - Public Sector: Government-funded in India but underfunded, with only 1.28% of GDP allocated to health.
   

In [17]:
# guide user next query; it contains the clarified question and some proposed guiding questions

guide = agent4.get_response(user_content=input)['response_content']

In [18]:
print(guide)

A refined question that clarifies your query and additional questions for your future thought are given as follows:

Clarified Question:
"How can the National Medical Council (NMC) address the expanding disparity between the public and private health sectors in India, especially considering the increased dependence on Auxiliary Nurse Midwives (ANMs) and Accredited Social Health Activists (ASHAs) for rural healthcare, and concurrently tackle the challenges of integrating digital technology and enhancing health infrastructure as outlined under the National Health Mission (NHM)?"

Proposed Guiding Questions:
1. "In what specific ways can the NMC leverage its regulatory powers to ensure equitable healthcare delivery between the public and private sectors, particularly in rural areas served by ANMs and ASHAs?"
2. "How might the NMC utilize the increased federal health budget to bridge the gap between public and private healthcare services and support the NHM's objectives?"
3. "What strategi

#### The asnwer above is more comprehensive; but more tunnings will go on...